In [1]:
#imports necessários
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
format = 'parquet'
inferSchema = 'false'
header = 'false'
multiline = 'false'

In [4]:
#Configurando spark
conf = SparkConf().setAppName('Load').set("spark.serializer", "org.apache.spark.serializer.KryoSerializer").set("hive.exec.dynamic.partition", "true").set("hive.exec.dynamic.partition.mode", "nonstrict")
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()


In [5]:
#definindo caminho 
filePath1 = 'C:\\Users\\dbello\\Documents\\pd-desafio\\refined_data\\baseA\\*.parquet'
filePath2 = 'C:\\Users\\dbello\\Documents\\pd-desafio\\raw_data\\baseB\\*.parquet'

In [6]:
#Lendo parquet para df
df_tratado = spark.read.format(format) \
                    .option("inferSchema", inferSchema) \
                    .option("header", header) \
                    .option("multiLine", multiline) \
                    .parquet(filePath1)
df_baseB = spark.read.format(format) \
                    .option("inferSchema", inferSchema) \
                    .option("header", header) \
                    .option("multiLine", multiline) \
                    .parquet(filePath2)

In [7]:
#Adicionando coluna de id puro na base B
df2 = df_baseB.withColumn("studentId", df_baseB.studentId_clientType.substr(1, 64))
 

In [8]:
#fazendo join das duas bases a partir do idstudent
df_final = df2.join(df_tratado, df2.studentId == df_tratado.Id, how = 'left').drop(df2.studentId)

In [9]:
#df final 
df_final.limit(50).toPandas()

,LastAccessedUrl,PageCategory,PageCategory1,PageCategory2,PageCategory3,PageName,at,browser,carrier,city_name,clv_total,country,custom_1,custom_2,custom_3,custom_4,device_new,first-accessed-page,install_uuid,language,library_ver,marketing_campaign,marketing_medium,marketing_source,model,name,nth,os_ver,platform,region,session_uuid,studentId_clientType,type,user_type,uuid,City,Id,RegisteredDate,SignupSource,State,StudentClientSignUp,CourseName,UniversityName,PaymentDate,PlanType,FollowDate,SubjectName,SessionStartTime,StudentClientSession
0,None,disciplina,disciplina,probabilidade-e-estatistica,Undefined,/disciplina/probabilidade-e-estatistica?type=1...,2017-11-16 00:02:34,Chrome 62,Itline Telecom,None,None,br,UFCG,Engenharia Elétrica,commerce,Core User,False,arquivo,2cc3fe9942cfc1f8a11551f5c5d608ef50ade273,pt,web_3.3.3,None,None,None,MacIntel,Page View,68,10.13.1,Mac OS X,None,bebcd366565bd5eaa0327007e5439fd9d2edef5f,01f26388ece24bead577f3829976d40e520235b7daf7b3...,e,known,a0a56ef62885cf1c77ccca15aca682ac43f85ce3,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,cadastro,None,None,None,/cadastro/passo2,2017-11-16 19:41:11,Mobile Safari 11,Copel Telecomunicacoes S.a.,None,None,br,None,None,None,None,True,None,cf6e513b84c300f8a4c4484c8721fb29a0077ffe,pt,web_3.3.3,None,None,None,iPhone,Page View,1,11.0.3,iOS,None,a20655393f63e233f1c2b5cabe37e1a461f13ff2,01fb1b445d05ab47b40c2877faae2b45366670ac1bef23...,e,known,ede40bc0209650f2eb3b450da123a68b1b73d9e6,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,home,None,None,None,/,2017-11-16 19:42:03,Mobile Safari 11,Copel Telecomunicacoes S.a.,None,None,br,None,None,None,None,True,None,cf6e513b84c300f8a4c4484c8721fb29a0077ffe,pt,web_3.3.3,None,None,None,iPhone,Page View,1,11.0.3,iOS,None,a20655393f63e233f1c2b5cabe37e1a461f13ff2,01fb1b445d05ab47b40c2877faae2b45366670ac1bef23...,e,known,12872b1725d2a50bb0c5c9dbf8acad4fce02a322,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,disciplina,disciplina,processo-civil,Undefined,/disciplina/processo-civil,2017-11-16 17:51:42,Chrome 61,Companhia Itabirana Telecomunicacoes Ltda,None,None,br,UFOP,Direito,commerce,Core User,False,arquivo,600febc0ddb87a192b163b657eac9df1bf97d794,pt,web_3.3.3,None,None,None,Win32,Page View,370,,Windows 10,None,92ee56d1eab22e395e30b3011f62479c00774b14,0268a86cb81655059d4508eb802304a3d58c78ea72a5f6...,e,known,707e9f32bfb7e9500af30b375789cd4b2700b8f0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,disciplina,disciplina,processo-civil-ii,Undefined,/disciplina/processo-civil-ii,2017-11-16 17:50:40,Chrome 61,Companhia Itabirana Telecomunicacoes Ltda,None,None,br,UFOP,Direito,commerce,Core User,False,arquivo,600febc0ddb87a192b163b657eac9df1bf97d794,pt,web_3.3.3,None,None,None,Win32,Page View,370,,Windows 10,None,92ee56d1eab22e395e30b3011f62479c00774b14,0268a86cb81655059d4508eb802304a3d58c78ea72a5f6...,e,known,6e3f8f2c28f622d87435c10dd89369cebabf5119,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,None,disciplina,disciplina,processo-civil,Undefined,/disciplina/processo-civil?type=6&materialid=3...,2017-11-16 17:23:15,Chrome 61,Companhia Itabirana Telecomunicacoes Ltda,None,None,br,UFOP,Direito,commerce,Core User,False,arquivo,600febc0ddb87a192b163b657eac9df1bf97d794,pt,web_3.3.3,None,None,None,Win32,Page View,370,,Windows 10,None,92ee56d1eab22e395e30b3011f62479c00774b14,0268a86cb81655059d4508eb802304a3d58c78ea72a5f6...,e,known,1ff7261b1d3278de28a0577abf05ea7b97f1743b,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,None,disciplina,disciplina,didAtica,Undefined,/disciplina/didAtica?type=6&materialid=3827927...,2017-11-16 00:10:37,Chrome 61,Wlenet Informatica Manutencao,None,None,br,ESTÁCIO,Marketing,None,Cold User,False,arquivo,4301274adfd9db1241c964650e489d57b9118bf1,pt,web_3.3.3,provas_ESTÁCIO EAD_2_2,midia,email_mkt,Win32,Page View,152,,Windows 7,None,114297d7e12225f9724

In [10]:
#Visão das páginas acessadas com o total de cada sessão
df = df_final.groupBy("PageName").agg(countDistinct("session_uuid")).show()

+--------------------+----------------------------+
|            PageName|count(DISTINCT session_uuid)|
+--------------------+----------------------------+
|/busca?q=anderson...|                           1|
|/disciplina/epide...|                           1|
|/arquivo/10753885...|                           1|
|/disciplina/inova...|                           4|
|/busca?q=resposta...|                           1|
|/disciplina/intro...|                           2|
|/disciplina/inter...|                           6|
|/pergunta/1649340...|                          11|
|/pergunta/3625085...|                           3|
|/arquivo/1020966/...|                           2|
|/disciplina/avali...|                          18|
|/disciplina/atenc...|                           2|
|/disciplina/topic...|                           1|
|/disciplina/gesta...|                          19|
|/busca?q=caracter...|                           1|
|/busca?q=avalia%C...|                           1|
|/lista/3398

In [11]:
#modelos mais utilizados
df = df_final.groupBy("model").agg(countDistinct("session_uuid")).orderBy(desc("count(DISTINCT session_uuid)")).show()

+--------------------+----------------------------+
|               model|count(DISTINCT session_uuid)|
+--------------------+----------------------------+
|               Win32|                      165622|
|        Linux armv7l|                       16986|
|               Win64|                        7562|
|              iPhone|                        6615|
|        Linux armv8l|                        4620|
|            MacIntel|                        2682|
|        Linux x86_64|                        1598|
|                iPad|                         601|
|          Linux i686|                         586|
|                 ARM|                         364|
|       Linux aarch64|                         294|
|Pike v8.0 release...|                         105|
|Linux i686 on x86_64|                           7|
|          BlackBerry|                           4|
|       PlayStation 4|                           2|
|Pike v7.8 release...|                           1|
|          i

In [12]:
#Categorias por sessão
df = df_final.groupBy("PageCategory", df_final.PageCategory2).agg(countDistinct("session_uuid")).orderBy(desc("count(DISTINCT session_uuid)")).show()

+--------------------+-------------+----------------------------+
|        PageCategory|PageCategory2|count(DISTINCT session_uuid)|
+--------------------+-------------+----------------------------+
|                home|    Undefined|                       41912|
|               busca|    Undefined|                       28152|
|             premium|         null|                       23659|
|            pergunta|         null|                       22093|
|            cadastro|    Undefined|                       18454|
|             arquivo|         null|                       15521|
|                home|         null|                       11573|
|              perfil|    Undefined|                       11412|
|               lista|    Undefined|                        8403|
|              listas|    Undefined|                        7929|
|           pagamento|    Undefined|                        5701|
|exercicios-resolv...|         null|                        3534|
|  explora

In [13]:
#Escrevendo arquivo parquet tratado
df_final.write.parquet('C:\\Users\\dbello\Documents\\pd-desafio\\refined_data\\baseB')